# Day16：魔法方法

在python中，有一些内置好的特定的方法，这些方法在进行特定的操作时会自动被调用，称之为魔法方法，下面介绍几种常见的魔法方法

##  \__init__方法

初始化函数，在创建实例对象为其赋值时使用，在\__new__之后，\__init__必须至少有一个参数self，就是这个\__new__返回的实例，\__init__是在\__new__的基础上可以完成一些其它初始化的动作，\__init__不需要返回值

## \__new__方法

很多人认为\__init__是类的构造函数，其实不太确切，\__init__更多的是负责初始化操作，相当于一个项目中的配置文件，\__new__才是真正的构造函数，创建并返回一个实例对象，如果\__new__只调用了一次，就会得到一个对象。继承自object的新式类才有\__new__这一魔法方法，\__new__至少必须要有一个参数cls，代表要实例化的类，此参数在实例化时由Python解释器自动提供，\__new__必须要有返回值，返回实例化出来的实例（很重要），这点在自己实现\__new__时要特别注意，可以return父类\__new__出来的实例，或者直接是object的\__new__出来的实例，若\__new__没有正确返回当前类cls的实例，那\__init__是不会被调用的，即使是父类的实例也不行。\__new__是唯一在实例创建之前执行的方法，一般用在定义元类时使用

创建对象的步骤：

a、首先调用\__new__得到一个对象

b、调用\__init__为对象添加属性

c、将对象赋值给变量

In [1]:
class A:
    pass
class B(A):
    def __init__(self):
        print('__init__方法被调用')
    def __new__(cls):
        print('__new__方法被调用')
        print(id(cls))
        return object.__new__(A)

b = B()
print(b)
print(type(b))
print(id(A))
print(id(B))
print(id(b))

__new__方法被调用
1839932171288
<class '__main__.A'>
1839932202440
1839932171288
1839943007688


从运行结果可以看出，\__new__中的参数cls和B的id是相同的，表明\__new__中默认的参数cls就是B类本身，而在return时，并没有正确返回当前类cls的实例，而是返回了其父类A的实例，因此\__init__这一魔法方法并没有被调用，此时\__new__虽然是写在B类中的，但其创建并返回的是一个A类的实例对象

下面我们改变\__new__的参数为cls再试一遍

In [2]:
class A:
    pass
class B(A):
    def __init__(self):
        print('__init__方法被调用')
    def __new__(cls):
        print('__new__方法被调用')
        print(id(cls))
        return object.__new__(cls)

b = B()
print(b)
print(type(b))
print(id(A))
print(id(B))
print(id(b))

__new__方法被调用
1839932204328
__init__方法被调用
<class '__main__.B'>
1839932203384
1839932204328
1839943043784


可以看出，当\__new__正确返回其当前类cls的实例对象时，\__init__被调用了，此时创建并返回的是一个B类的实例对象

## \__class__方法

\__class__方法作用是获取已知对象的类

In [3]:
print(b.__class__)

<class '__main__.B'>


\__class__至少在下面这种情况中是有用的：即当一个类中的某个成员变量是所有该类的对象的公共变量时，下面看一个例子

In [4]:
class C:
    count = 0
    def addcount(self):
        self.__class__.count += 1
        
a = C()
b = C()
a.addcount()
print(a.count)
b.addcount()
print(b.count)

1
2


## \__str__方法

在将对象转换成字符串 str (对象)测试的时候，打印对象的信息,\__str__方法必须要return一个字符串类型的返回值，作为对实例对象的字符串描述，\__str__实际上是被 print函数默认调用的，当要 print（实例对象）时，默认调用\__str__方法，将其字符串描述返回。如果不是要用str()函数转换。当你打印一个类的时候，那么print首先调用的就是类里面的定义的\__str__

In [5]:
class D:
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return('我的名字是%s' % self.name)
    def __repr__(self):
        return('这是D类的repr方法')
d = D('Leon')
print(d)
print(D)

我的名字是Leon
<class '__main__.D'>


## \__repr__方法

如果说\__str__体现的是一种可读性，是给用户看的，那么\__repr__方法体现的则是一种准确性，是给开发人员看的，它对应的是repr()函数，重构\__repr__方法后，在控制台直接敲出实例对象的名称，就可以按照\__repr__中return的值显示了

In [6]:
d

这是D类的repr方法

当我们想所有环境下都统一显示的话，可以重构\__repr__方法；当我们想在不同环境下支持不同的显示，例如终端用户显示使用\__str__，而程序员在开发期间则使用底层的\__repr__来显示，实际上\__str__只是覆盖了\__repr__以得到更友好的用户显示

## \__del__方法

对象在程序运行结束之后进行垃圾回收的时候调用这个方法，来释放资源。此时，此方法是被自动调用的。除非有特殊要求，一般不要重写。在关闭数据库连接对象的时候，可以在这里，释放资源

In [11]:
class E:
    num = 0
    def __init__(self, name):
        self.name = name
        E.num += 1
        print(self.name, E.num)
    def __del__(self):
        E.num -= 1
        print('del', self.name, E.num)
aa = E('Python')
bb = E('C++')
cc = E('java')
print('over')
del aa
del bb
del cc

Python 1
C++ 2
java 3
over
del Python 2
del C++ 1
del java 0


## \__getattribute__方法

属性访问拦截器，在访问实例属性时自动调用。在python中，类的属性和方法都理解为属性，且均可以通过\__getattribute__获取。当获取属性时，相当于对属性进行重写，直接return object.\__getattribute__(self, \*args, \**kwargs)或者根据判断return所需要的重写值，如果需要获取某个方法的返回值时，则需要在函数后面加上一个()即可。如果不加的话，返回的是函数引用地址。下面看一个例子

In [12]:
class F:
    def __init__(self, lang):
        self.sub1 = lang
        self.sub2 = 'Python'
    def __getattribute__(self, obj):
        if obj == 'sub1':
            return 'programming'
        else:
            return object.__getattribute__(self, obj)
s = F('C++')
print(s.sub1)
print(s.sub2)

programming
Python


## \__base__方法

获取指定类的所有父类构成元素，使用方法为类名\.\_\_bases\_\_

## \__mro__方法

显示指定类的所有继承脉络和继承顺序，假如这个指定的类不具有某些方法和属性，但与其有血统关系的类中具有这些属性和方法，则在访问这个类本身不具有的这些方法和属性时，会按照\__mro__显示出来的顺序一层一层向后查找，直到找到为止

## \__call__方法

具有\__call__魔法方法的对象可以使用XXX()的形式被调用，比如说类的实例对象

In [13]:
class H:
    def __init__(self):
        print('__init__被调用')
    def __call__(self):
        print('__call__被调用')
h = H()
h()

__init__被调用
__call__被调用


## \__all__方法

将一个py文件作为模块导入时，其中if \__name__ == "main"以上的类、方法、函数等都能被导入，但某些方法可能只是用来做测试用的，不希望也不建议被导入，可以用\__all__=['函数名或方法名']的方式限制一下哪些函数或方法可以被导入，即[]中的函数名或方法名可以被导入。但是需要强调的是，\__all__魔法方法只针对通过 from xx import *这种导入方式有效

```Python
__all__ = ['a', 'b']
def a():
    pass
def b():
    pass
def c():
    pass
----------------
from file.py import *
a()
b()
c() #产生错误
```